In [52]:
import pandas as pd
import numpy as np
import glob
import re

In [58]:
def check_sim_v2(check_word, sims):
    if re.compile('m[a-z]+sq[a-z]+to').match(check_word):
        sims.append(check_word)
        return True, sims
    
    words = ['repel', 'protect', 'spray', 
             "proteger", "repeler", "rociar","repelir"]

    for word in words:
        if re.compile(word).search(check_word):
            sims.append(check_word)
            return True, sims
    
    return False, sims

In [92]:
file = "/Users/datacsv/tweets_2016_02_08.csv"

In [113]:
temp_df = pd.read_csv("/Users/datacsv/tweets_2015_08_16.csv")
filtered_df = pd.DataFrame(columns=temp_df.columns)
data = {}
regex = re.compile('[^a-zA-Z]')
index = 0
sims = []

df = pd.read_csv(file)
df[df == "None"] = np.nan
df = df.dropna(subset=["City", "State", "Country"], how='all')
df = df.fillna("")


#Find number of total tweets in a location
group_sums = df.groupby(["City", "State", "Country"])["Message"].agg(np.size)


cities = group_sums.index.get_level_values(0)
states = group_sums.index.get_level_values(1)
countries = group_sums.index.get_level_values(2)

for city, state, country in zip(cities, states, countries):
    location = (city, state, country)
    data[location] = data.get(location, {})
    data[location]['num_total'] = data[location].get('num_total', 0)
    data[location]['num_total'] += group_sums.loc[city, state, country]

#Filter tweets, then store in filtered_df
for i in df.index:
    tweet = df.loc[i]["Message"]
    if tweet[0:2] != "RT":
        for word in tweet.lower().split():
                isSimilar, sims = check_sim_v2(regex.sub('', word), sims)
                if(isSimilar):
                    filtered_df.loc[index] = df.loc[i]
                    index += 1
                    break
                    

In [114]:
group_sums = filtered_df.groupby(["City", "State", "Country"])["Message"].agg(np.size)
cities = group_sums.index.get_level_values(0)
states = group_sums.index.get_level_values(1)
countries = group_sums.index.get_level_values(2)

for city, state, country in zip(cities, states, countries):
    location = (city, state, country)
    data[location] = data.get(location, {})
    data[location]['filtered_total'] = data[location].get('filtered_total', 0)
    data[location]['filtered_total'] += group_sums.loc[city, state, country]
    

In [115]:
final_df = pd.DataFrame(columns=['city', 'state', 'country', 'total_tweets', 
                                 'filtered_tweets', 'percentage'])
index = 0
for col in data:
    try:
        total_tweets = data[col]['num_total']
    except KeyError:
        continue
        
    try:
        filtered_tweets = data[col]['filtered_total']
    except KeyError:
        filtered_tweets = 0
        
    percentage = filtered_tweets/total_tweets * 100
    final_df.loc[index] = [col[0], col[1], col[2], total_tweets, filtered_tweets,
                          percentage]
    index += 1
    

In [116]:
final_df[final_df['percentage'] > 100]

,city,state,country,total_tweets,filtered_tweets,percentage


In [117]:
df[df['City'] == 'Antioch']

,Tweet_ID,Message,Date_Posted,Language,Gender,Coordinates_From_Metadata_or_Carmen,Latitude,Longitude,City,State,Country
15494,696533846725033984,My little cousin SCREAM he a boy because he sa...,2016-02-08T03:19:44.000Z,en,M,c,38.015825,-121.819699,Antioch,California,United States


In [118]:
filtered_df[filtered_df['City'] == 'Antioch']

,Tweet_ID,Message,Date_Posted,Language,Gender,Coordinates_From_Metadata_or_Carmen,Latitude,Longitude,City,State,Country
173,696533846725033984,My little cousin SCREAM he a boy because he sa...,2016-02-08T03:19:44.000Z,en,M,c,38.015825,-121.819699,Antioch,California,United States
